In [1]:
using Dates
import PlotlyJS

include("../Sensors/Sensors.jl")
include("../PreProcessing/PreProcessing.jl")
include("../Utils/Helpers.jl")
include("../Utils/Visualizations.jl")
using .Sensors, .PreProcessing, .Visualizations, .Helpers;

WebIO._IJuliaInit()

## Load the data and format it

In [2]:
#= datafolder = "../../Data/Twitter/"
datafile = datafolder * "all_links_processed.json"
data = load_json(datafile) =#

datafolder = "../../Data/Twitter/COP26_processed_lightweight/"
datafiles = [file for file in readdir(datafolder) if occursin(".json", file)]
frames = [load_json(datafolder * file) for file in datafiles]
data = vcat(frames...)

data = data[.~ismissing.(data."domain"), :]

to_datetime = x -> DateTime(split(x, '.')[1], "yyyy-mm-ddTHH:MM:SS")
data."created_at" = to_datetime.(data."created_at");

LoadError: UndefVarError: DataFrame not defined

## Define the partition, actors and actions

In [ ]:
part_fun = cop_26_dates
actor_fun = follower_count
action_fun = trust_score

df = data |> part_fun |> action_fun |> actor_fun

actions = sort(unique(df.action))
actors = sort(unique(df.actor))
partitions = sort(unique(df.partition));

## Statistics on the data

In [ ]:
show(plot_actor_frequency(df))

In [ ]:
plot_actor_wordcloud(df, Nactor=300)

In [ ]:
show(plot_action_frequency(df))

## Compute influence graph

In [ ]:
hours = 5
minutes = 0;

In [ ]:
tsg = TimeSeriesGenerator()
# ig = InfluenceGraphGenerator()
ig = InfluenceGraphGenerator(SMeasure);

In [ ]:
total_min = hours*60 + minutes

clean_dates = x -> floor(x, Dates.Minute(total_min))
df.time = clean_dates.(df."created_at")

time_series = observe(df, tsg)
influence_graph = observe(time_series, ig);

## Compute influence cascade

In [ ]:
cuttoff = 0.5

In [ ]:
icg = InfluenceCascadeGenerator(cuttoff)
influence_cascades = observe.(influence_graph, Ref(icg))

edge_types = [string(n1, " to ", n2) for n1 in actions for n2 in actions]
push!(edge_types, "Any Edge")

## Plot influence graph

In [ ]:
partition = "During COP26"
edge = "Any Edge"

In [ ]:
simplifier = make_simplifier(edge, cuttoff, edge_types)
partition_index = (1:length(partitions))[findfirst(partition .== partitions)]
	
# In this case we plot the graph on a world map
if actor_fun == country
	PlotlyJS.plot(map_plot(df)...)
# In this case we plot a simple graph of the actors
else
	plot_graph(influence_graph[partition_index], df, simplifier=simplifier)
end

## Plot cascades and statistics

In [ ]:
influencer_indices = [ic.root for ic in influence_cascades[partition_index]]
influencers = actors[influencer_indices]

In [ ]:
influencer_node1 = influencers[1]

PlotlyJS.plot(plot_cascade_sankey(influence_cascades[partition_index][findfirst(influencer_node1 .== influencers)], df)...)

In [ ]:
show(plot_actors_per_level(influence_cascades, df))